In [ ]:
import pandas as pd
df = pd.read_csv('Lega_PerSum.csv')
df.head()

In [ ]:
# pose sequence as a NLI premise and label as a hypothesis
from transformers import AutoModelForSequenceClassification, AutoTokenizer
nli_model = AutoModelForSequenceClassification.from_pretrained('facebook/bart-large-mnli').to('cuda')
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-mnli')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
import numpy as np
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

premise = df["Judgement"].to_list()
hypothesis = df["Perspective-based Summary"].to_list()

In [ ]:
from tqdm import tqdm

# Initialize the dictionary
NLI_Judgement_Summary = {
    "Judgement": [],
    "Perspective-based Summary": [],
    "Contradiction": [],
    "Neutral": [],
    "Entailment": [],
}

# Loop through the premise (judgments) and hypothesis (summaries) with tqdm for progress tracking
for i, (p, h) in tqdm(enumerate(zip(premise, hypothesis)), total=len(premise)):
    # Encode the premise and hypothesis for NLI
    x = tokenizer.encode(p, h, return_tensors='pt', truncation=True)

    # Get the logits from the model and calculate softmax probabilities
    logits = nli_model(x.to(device))[0]
    probs = logits.softmax(dim=1)

    # Extract probabilities for each class: contradiction, neutral, entailment
    prob_contradiction = probs[:, 0].item()
    prob_neutral = probs[:, 1].item()
    prob_entailment = probs[:, 2].item()

    # Append values to the dictionary
    NLI_Judgement_Summary["Judgement"].append(p)
    NLI_Judgement_Summary["Perspective-based Summary"].append(h)
    NLI_Judgement_Summary["Contradiction"].append(prob_contradiction)
    NLI_Judgement_Summary["Neutral"].append(prob_neutral)
    NLI_Judgement_Summary["Entailment"].append(prob_entailment)

# Now the NLI_Judgement_Summary dictionary will contain the judgments, summaries, and their corresponding NLI scores

In [ ]:
df = pd.DataFrame(NLI_Judgement_Summary)

In [ ]:
entailment_count = (df['Entailment'] > 0.60).sum()
contradiction_count = (df['Contradiction'] < 0).sum()
neutral_count = ((df['Neutral'] > 0.3) & (df["Neutral"] < 0.6)).sum()

# Printing the NLI Scores
print(entailment_count/len(df)), print(contradiction_count/len(df)), print(neutral_count / len(df))